In [1]:
!pip install folium

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import geopandas as gpd
import json
import folium
from shapely.geometry import shape
import plotly.express as px

In [3]:
df_final=pd.read_excel('SucRates.xlsx')
df_final['Location'] = df_final['Location'].str.upper().str.strip()
df_final.head()

,Location,Distance,Total_AIs,Successful_AIs,Success_rate
0,"ABORLAN,PALAWAN",609.665498,65,0,0.000000
1,"ABRA DE ILOG,OCCIDENTAL MINDORO",115.136113,96,20,0.208333
2,"ABUYOG,LEYTE",23.544343,30,10,0.333333
3,"AGDANGAN,QUEZON",81.692935,19,2,0.105263
4,"AGLIPAY,QUIRINO",119.900207,885,173,0.195480


In [4]:

# If using Jupyter or another environment, replace 'path/to/yourfile.geojson' with your GeoJSON file path
geojson_file = 'https://geodata.ucdavis.edu/gadm/gadm4.1/json/gadm41_PHL_2.json'

# Step 2: Read the GeoJSON Data
import geopandas as gpd

# Download and read the GeoJSON data
gdf = gpd.read_file(geojson_file)
gdf.head()


,GID_2,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry
0,PHL.1.1_1,PHL,Philippines,PHL.1_1,Abra,NA,Bangued,NA,NA,Bayan|Munisipyo,Municipality,140101,PH.AB.BN,"MULTIPOLYGON (((120.61130 17.49560, 120.60550 ..."
1,PHL.1.2_1,PHL,Philippines,PHL.1_1,Abra,NA,Boliney,NA,NA,Bayan|Munisipyo,Municipality,140102,PH.AB.BL,"MULTIPOLYGON (((120.81400 17.34210, 120.76860 ..."
2,PHL.1.3_1,PHL,Philippines,PHL.1_1,Abra,NA,Bucay,NA,NA,Bayan|Munisipyo,Municipality,140103,PH.AB.BU,"MULTIPOLYGON (((120.67770 17.45740, 120.67320 ..."
3,PHL.1.4_1,PHL,Philippines,PHL.1_1,Abra,NA,Bucloc,NA,NA,Bayan|Munisipyo,Municipality,140104,PH.AB.BC,"MULTIPOLYGON (((120.78920 17.44520, 120.88460 ..."
4,PHL.1.5_1,PHL,Philippines,PHL.1_1,Abra,NA,Daguioman,NA,NA,Bayan|Munisipyo,Municipality,140105,PH.AB.DG,"MULTIPOLYGON (((120.90750 17.41930, 120.89280 ..."


In [5]:
gdf['Location'] = gdf['NAME_2'] + "," + gdf['NAME_1']
gdf['Location'] = gdf['Location'].str.upper().str.strip()
gdf.drop(columns=['GID_2','GID_0','COUNTRY','GID_1','NAME_1','NL_NAME_1','NAME_2','VARNAME_2','NL_NAME_2','TYPE_2','ENGTYPE_2','CC_2','HASC_2'], inplace=True)
gdf.head()

,geometry,Location
0,"MULTIPOLYGON (((120.61130 17.49560, 120.60550 ...","BANGUED,ABRA"
1,"MULTIPOLYGON (((120.81400 17.34210, 120.76860 ...","BOLINEY,ABRA"
2,"MULTIPOLYGON (((120.67770 17.45740, 120.67320 ...","BUCAY,ABRA"
3,"MULTIPOLYGON (((120.78920 17.44520, 120.88460 ...","BUCLOC,ABRA"
4,"MULTIPOLYGON (((120.90750 17.41930, 120.89280 ...","DAGUIOMAN,ABRA"


In [6]:
gdf_final=gdf.merge(df_final, on='Location', how='inner')

In [7]:
gdf_final.head()

,geometry,Location,Distance,Total_AIs,Successful_AIs,Success_rate
0,"MULTIPOLYGON (((120.61130 17.49560, 120.60550 ...","BANGUED,ABRA",50.602129,44,22,0.500000
1,"MULTIPOLYGON (((120.67770 17.45740, 120.67320 ...","BUCAY,ABRA",59.276756,40,15,0.375000
2,"MULTIPOLYGON (((120.74980 17.62310, 120.74890 ...","DOLORES,ABRA",47.289538,41,30,0.731707
3,"MULTIPOLYGON (((120.71870 17.58940, 120.71600 ...","LAGANGILANG,ABRA",52.016533,69,39,0.565217
4,"MULTIPOLYGON (((120.59740 17.31130, 120.57810 ...","PILAR,ABRA",70.427789,222,81,0.364865


In [11]:

#  Convert GeoDataFrame to GeoJSON
geojson_data = gdf_final.__geo_interface__

# Step 4: Create the Choropleth Map
m = folium.Map(location=[12.8797, 121.7740], zoom_start=5.5)  # Centered on the Philippines

# Add the color scale as a legend to the map
#linear.add_to(m)

folium.Choropleth(
    geo_data=geojson_data,  # GeoJSON data
    name="choropleth",
    data=df_final,  # DataFrame with data to visualize
    columns=['Location', 'Success_rate'],  # Columns from the DataFrame
    key_on='feature.properties.Location',  # Key in GeoJSON file to match with 'LOCATION'
    choropleth_data=df_final,
    fill_color='YlOrRd',  # Color scheme
    nan_fill_color='black',
    nan_fill_opacity=0.3,
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Success Rate of Artificial Insemination per Distance',
).add_to(m)


# Add markers to the map with popups
#for _, row in df_WVSU_success_rates.iterrows():
#    folium.Marker(
#        location=[row['MUN_LAT'], row['MUN_LONG']],
#        popup=folium.Popup(
#            f"Municipality: {row['LOCATION']}",
#            max_width=300
#        ),
#        tooltip=row['LOCATION']
#    ).add_to(m)

# Add marker for the center (WVSU location)
folium.Marker(
    location=[10.59718635, 123.3998131],  # Approximate coordinates for WVSU
    popup='DA-PCC LCSF',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

folium.Marker(
    location=[7.974721199, 122.5816288],  # Approximate coordinates for WVSU
    popup='DA-PCC MLPC',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

folium.Marker(
    location=[15.7419217, 120.9416352],  # Approximate coordinates for WVSU
    popup='DA-PCC HEADQUARTERS',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

folium.Marker(
    location=[10.97513811, 124.742286],  # Approximate coordinates for WVSU
    popup='DA-PCC VSU',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

folium.Marker(
    location=[14.1588833, 121.2446535],  # Approximate coordinates for WVSU
    popup='DA-PCC UPLB',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

folium.Marker(
    location=[7.8566615, 125.0496225],  # Approximate coordinates for WVSU
    popup='DA-PCC CMU',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

folium.Marker(
    location=[11.11896418, 122.5531773],  # Approximate coordinates for WVSU
    popup='DA-PCC WVSU',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

folium.Marker(
    location=[7.112249738, 124.8416306],  # Approximate coordinates for WVSU
    popup='DA-PCC USM',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

folium.Marker(
    location=[16.2920167, 120.4269356],  # Approximate coordinates for WVSU
    popup='DA-PCC DMMMSU',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

folium.Marker(
    location=[9.994206418, 124.4488381],  # Approximate coordinates for WVSU
    popup='DA-PCC USF',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

folium.Marker(
    location=[18.15759136, 120.5549955],  # Approximate coordinates for WVSU
    popup='DA-PCC MMSU',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)
# Display the map
m

# The GeoJSON file for Philippine municipalities is now directly downloaded and used
# You don't need to save it separately unless required for other purposes

# Display the first few rows of the GeoDataFrame to verify the data
display(m)

# Save the map
m.save('SucRates_map.html')
